<a href="https://colab.research.google.com/github/SNJ292/snj292.github.io/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

years = list(range(2012, 2022))
url_start = "https://www.basketball-reference.com/draft/NBA_{}.html"

for year in years:
  url = url_start.format(year)
  data = requests.get(url)

  with open("draft/{}.html".format(year), "w+")as f:
    f.write(data.text)

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

with open("draft/2012.html") as f:
  page = f.read()

soup = BeautifulSoup(page, "html.parser")
soup.find('tr', class_="over_header").decompose
draft_table = soup.find(id="stats")
draft_2012 = pd.read_html(str(draft_table))[0]
draft_2012.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                  Rk                 Pk                 Tm   
0                  1                  1                NOH   
1                  2                  2                CHA   
2                  3                  3                WAS   
3                  4                  4                CLE   
4                  5                  5                SAC   

                  Round 1           Unnamed: 5_level_0 Totals                \
                   Player   College                Yrs      G     MP    PTS   
0           Anthony Davis  Kentucky                 11    624  21497  14962   
1  Michael Kidd-Gilchrist  Kentucky                  8    446  10978   3750   
2            Bradley Beal   Florida                 11    663  23049  14643   
3            Dion Waiters  Syracuse                  8    419  11835   5505   
4         Thomas Robinson    Kansas                  5    313   4204   1528   

         ... Shooting       Per Game                  Advanced              \
    TRB  ...      3P%   FT%       MP   PTS   TRB  AST       WS WS/48   BPM   
0  6419  ...     .304  .795     34.5  24.0  10.3  2.3     95.4  .213   6.1   
1  2420  ...     .272  .715     24.6   8.4   5.4  1.2     21.0  .092  -1.8   
2  2708  ...     .372  .822     34.8  22.1   4.1  4.2     50.5  .105   1.8   
3  1108  ...     .346  .694     28.2  13.1   2.6  2.8      8.8  .036  -2.1   
4  1507  ...     .000  .505     13.4   4.9   4.8  0.6      4.6  .053  -3.5   

         
   VORP  
0  44.0  
1   0.6  
2  21.7  
3  -0.4  
4  -1.6  

[5 rows x 22 columns]

In [ ]:
dfs = []
for year in years:
  with open("draft/2012.html") as f:
    page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_="over_header").decompose
    draft_table = soup.find(id="stats")
    draft = pd.read_html(str(draft_table))[0]
    draft.head()

    dfs.append(draft)
dfs

[   Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                    Rk                 Pk                 Tm   
 0                   1                  1                NOH   
 1                   2                  2                CHA   
 2                   3                  3                WAS   
 3                   4                  4                CLE   
 4                   5                  5                SAC   
 ..                ...                ...                ...   
 57                 56                 56                TOR   
 58                 57                 57                NJN   
 59                 58                 58                MIN   
 60                 59                 59                SAS   
 61                 60                 60                LAL   
 
                    Round 1           Unnamed: 5_level_0 Totals                \
                     Player   College                Yrs      G     MP    PTS   
 0  